In [ ]:
!pip install sentence_transformers
!pip install konlpy

In [ ]:
import numpy as np
import itertools

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [ ]:
doc = """애플이 정부의 지원을 받는 해킹에서 저명 인사를 보호하기 위해 올가을 아이폰과 아이패드, 맥 컴퓨터 등에 '록다운 모드'를 도입한다고 밝혔다고 경제매체 CNBC가 6일(현지시간) 보도했다.
록다운 모드가 되면 아이폰에서 일부 기능이 꺼져 해커가 접근하거나 해킹할 수 있는 기능이 크게 줄면서 아이폰이 스파이웨어에 덜 취약해진다.
구체적으로는 아이메시지의 미리 보기 기능, 애플의 웹브라우저인 사파리의 자바스크립트 제한, 신규 설정 프로필 설치 차단, 유선 연결 차단, 화상통화인 페이스타임 등 수신형 서비스 요청 차단 등이다.
애플은 앞서 작년 9월 아이폰과 맥 컴퓨터 등에 중대한 보안 취약점이 있다며 긴급 소프트웨어 업데이트를 내놓은 바 있다.
이스라엘의 보안기업 NSO그룹이 만든 스파이웨어 '페가수스'를 이용하면 해커들이 클릭 한번 없이도 애플 기기를 감염시켜 카메라나 마이크를 켜고 검색 기록, 문자 메시지나 이메일 내용 등이 노출될 수 있다는 이유에서였다.
페가수스 같은 고도의 스파이웨어는 가격이 수천억원에 달해 평범한 해커가 아닌 정부나 경찰 기관 등이 주요 고객이다.
애플은 이 사건 뒤 미국 의회와 각국 정부에서 이 문제를 해결하라는 압력을 받아왔다.
록다운 모드는 국가의 지원을 받는 해커의 표적이 될 수 있어 극도로 고도의 보안 수준이 요구되는 소수의 이용자를 위한 것이다. 여기에는 고위 정치인이나 언론인, 인권 활동가, 기업 임원 등이 포함된다.
애플은 "대다수 이용자는 고도의 사이버 공격의 희생자가 될 일이 결코 없겠지만 희생자가 될 수 있는 소수의 사람을 보호하기 위해 쉬지 않고 일하겠다"고 밝혔다.
애플은 록다운 모드에 혹시 있을지 모를 보안상 허점을 발견하는 사람에게 최고 200만달러(약 26억원)를 지급하겠다며 '현상금'까지 내걸었다."""

In [ ]:
def candidates_fn(doc, n_gram_range):
  okt = Okt()

  tokenized_doc = okt.pos(doc)
  tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])

  count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
  candidates = count.get_feature_names_out()

  return candidates

In [ ]:
def embedding_fn(doc, candidates):
  #이제 문서와 문서로부터 추출한 키워드들을 SBERT를 통해서 수치화한다
  model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

  doc_embedding = model.encode([doc])
  candidate_embeddings = model.encode(candidates)

  return (doc_embedding, candidate_embeddings)

In [ ]:
def keywords_extract(doc_embedding, candidate_embeddings, candidates, top_n):
  distances = cosine_similarity(doc_embedding, candidate_embeddings)
  keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]

  return keywords

In [ ]:
def max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n, nr_candidates):
  #문서와 각 키워드들 간의 유사도
  distances = cosine_similarity(doc_embedding, candidate_embeddings)
  #각 키워드들 간의 유사도
  distances_candidates = cosine_similarity(candidate_embeddings, candidate_embeddings)
  #코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick
  words_idx = list(distances.argsort()[0][-nr_candidates:])
  words_vals = [candidates[index] for index in words_idx]
  distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

  #각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
  min_sim = np.inf
  candidate = None
  for combination in itertools.combinations(range(len(words_idx)), top_n):
    sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
    if sim < min_sim:
      candidate = combination
      min_sim = sim

  return [words_vals[idx] for idx in candidate]

In [ ]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):
  #문서와 각 키워드들 간의 유사도가 적힌 리스트
  word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)
  #각 키워드들 간의 유사도
  word_similarity = cosine_similarity(candidate_embeddings, candidate_embeddings)
  #문서와 가장 높은 유사도를 가진 키워드의 인덱스 추출
  keywords_idx = [np.argmax(word_doc_similarity)]
  #가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
  candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]
  #최고의 키워드는 이미 추출. top_n-1번만큼 아래를 반복
  for _ in range(top_n-1):
    candidate_similarities = word_doc_similarity[candidates_idx, :]
    target_similarities = np.max(word_similarity[candidates_idx][:/, keywords_idx], axis=1)

    #MMR 계산
    mmr = (1-diversity)*candidate_similarities - diversity*target_similarities.reshape(-1,1)
    mmr_idx = candidates_idx[np.argmax(mmr)]

    #keywords & candidates 업데이트
    keywords_idx.append(mmr_idx)
    candidates_idx.remove(mmr_idx)

  return [words[idx] for idx in keywords_idx]

In [ ]:
candidates = candidates_fn(doc, (2,3))

In [ ]:
doc_embedding, candidate_embeddings = embedding_fn(doc, candidates)

In [ ]:
keywords = keywords_extract(doc_embedding, candidate_embeddings, candidates, 5)
keywords

['애플 웹브라우저', '보호 위해 애플', '애플 정부 지원', '아이폰 스파이웨어 취약', '아이폰 컴퓨터 보안']

In [ ]:
mss_keywords = max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=20) #높으면 더 다양
mss_keywords

['아이폰 아이패드 컴퓨터', '해커 정부 경찰', '컴퓨터 보안 취약점', '애플 앞서 작년', '보호 위해 애플']

In [ ]:
mmr_keywords = mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.7)  #높으면 더 다양
mmr_keywords

['아이폰 컴퓨터 보안', '각국 정부 문제', '정치인 언론인 인권', '기기 감염', '기업 임원']